<a href="https://colab.research.google.com/github/dan-a-iancu/airm/blob/master/Alset_Motors/Alset_Motors_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

____
# Basic Setup

Import useful modules, read the data and store it in data frames, and set up some useful Python lists. You may want to expand this section and make sure you understand how the data is organized, and also read the last part where the Python lists are created, as these may be very useful when you build your model.

In [ ]:
#@markdown We first import some useful modules. 

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# import numpy
import numpy as np
import urllib.request  # for file downloading

# Import pandas for data-frames
import pandas as pd
pd.options.display.max_rows = 15
pd.options.display.float_format = "{:,.2f}".format

from IPython.display import display

# Make sure Matplotlib runs inline, for nice figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import matplotlib.ticker as ticker 

# install Gurobi (our linear optimization solver)
!pip install -i https://pypi.gurobi.com gurobipy
from gurobipy import *

# some modules to create local directories for CBC (to avoid issues with solving multiple models)
import os
def new_local_directory(name):
    full_path = os.path.join(".", name)
    os.makedirs(full_path, exist_ok=True)
    return full_path

# install the latest version of seaborn for nicer graphics
#!pip install --prefix {sys.prefix} seaborn==0.11.0  &> /dev/null
import seaborn as sns

# Ignore useless some warnings
import warnings
warnings.simplefilter(action="ignore")

print("Completed successfully!")

Looking in indexes: https://pypi.gurobi.com
Completed successfully!


## Load the case data into Pandas data frames

We first download an Excel file with all the data from Github.

In [ ]:
#@markdown Download the entire data as an Excel file from Github

url_Excel = 'https://raw.githubusercontent.com/dan-a-iancu/airm/master/Alset_Motors/Alset_Motors_Data.xlsx?raw=true'
local_file = "Alset_Motors_Data.xlsx"   # name of local file where you want to store the downloaded file
urllib.request.urlretrieve(url_Excel, local_file)    # download from website and save it locally

('Alset_Motors_Data.xlsx', <http.client.HTTPMessage at 0x7f6e88dd3dd0>)

Read in and store the data in suitable dataframes.

In [ ]:
#@markdown Create dataframes based on the Excel file

# data on factories (Table 1); "index_col=0" just assigns the first column as the row names of the data frame
factoriesDF = pd.read_excel("Alset_Motors_Data.xlsx", sheet_name = "Factories", index_col=0)
display(factoriesDF)

# costs from factories to customization centers (Table 2)
costsFactoriesToCentersDF = pd.read_excel("Alset_Motors_Data.xlsx", \
                                    sheet_name = "Factories_to_Centers", index_col=0) 
display(costsFactoriesToCentersDF)

# customization center data (Table 3)
customizationCentersDF = pd.read_excel("Alset_Motors_Data.xlsx", sheet_name = "Customization_Centers", index_col=0)
display(customizationCentersDF)

# customer pre-order data (Table 4)
retailersDF = pd.read_excel("Alset_Motors_Data.xlsx", sheet_name = "Retailer_Preorders", index_col=0)
display(retailersDF)

# costs from customization centers to clients (Table 5)
costsCentersToRetailersDF = pd.read_excel("Alset_Motors_Data.xlsx", \
                              sheet_name = "Centers_to_Retailers", index_col=0)
display(costsCentersToRetailersDF)

,Cost,Capacity
Factory,,
Zuffe,54000,5000
Leip,58500,2000


,C1,C2,C3
Zuffe,1000,2500,2000
Leip,2000,1500,1800


,Staffing_Cost,Capacity
C1,3000,2000
C2,6000,2200
C3,7500,3300


,Pre-orders,Price
OO,5000,71000
FL,2500,74000
SH,1600,69000
WW,550,71000


,OO,FL,SH,WW
C1,4000,6000,3000,5000
C2,6000,6000,2000,4500
C3,4500,7000,6000,3000


## Create Python lists based on the data-frames

__NOTE__: Make sure you understand what the __lists__ created here are! These will be very helpful when creating the model.

In [ ]:
#@markdown Create useful lists with the factories, centers, clients
# Calculate some useful lists: the factories, the customization centers, the clients
factories = list( factoriesDF.index )              # the list of factories
centers = list( customizationCentersDF.index )     # the list of customization centers
retailers = list( retailersDF.index )              # the list of retailers

# Take a look at the lists
print("The list of factories:")
print(factories)

print("The list of centers:")
print(centers)

print("The list of retail clients:")
print(retailers)

The list of factories:
['Zuffe', 'Leip']
The list of centers:
['C1', 'C2', 'C3']
The list of retail clients:
['OO', 'FL', 'SH', 'WW']


<font color=red>**IMPORTANT HINT.**</font> When defining your **decision variables** in this model, you may want to consider *matrices* (or grids) of decision variables instead of a simple list like we've created before. Specifically, you may want to add a decision variable for every pair of elements where the first element is from one list and the second element is from another list. 

When the decision variables are organized in a grid, it's significantly easier to use the ``addVars`` function to add all of the decision variables to the model at once. The function ``addVars`` can take as arguments several lists, e.g., `addVars(list1, list2, list3,...)`, in which case it will return one decision variable for every possible tuple of values. These decision variables will be stored as a dictionary, with the dictionary keys corresponding to the tuple of values. 

For instance, suppose `list1 = ["Paris", "Budapest", "Beijing"]` and `list2= [1,2]`. Then `myDecisions = addVars(list1,list2)` would return a dictionary containing one decision for every pair, i.e., ("Paris",1), ("Paris",2), ("Budapest",1), ("Budapest",2), etc. To access the decision corresponding to the pair ("Paris",1), we can simply use `myDecisions[("Paris",1)]` or `myDecisions["Paris",1]`.